### 0. Data preparation

In [65]:
import numpy as np

# Load mnist.npz in root folder
data = np.load("mnist.npz")
X_train, Y_train = data["x_train"], data["y_train"]
X_test, Y_test = data["x_test"], data["y_test"]
X_train = X_train.reshape(X_train.shape[0], -1) / 255
X_test = X_test.reshape(X_test.shape[0], -1) / 255

# Split ratio of trainand and validation as 5:1
TSIZE = 50000
X_train_ = X_train[:TSIZE]
X_valid_ = X_train[TSIZE:]
Y_train_ = Y_train[:TSIZE]
Y_valid_ = Y_train[TSIZE:]

X_train = X_train_
X_valid = X_valid_
Y_train = Y_train_
Y_valid = Y_valid_

print(X_train.shape)
print(X_valid.shape)
print(Y_train.shape)
print(Y_valid.shape)

(50000, 784)
(10000, 784)
(50000,)
(10000,)


### 1. Building the Model

In [62]:
# Dimens of MNIST input
N_INPUT = 28*28
# Total number of class
N_CLASSES = 10

# 2 hidden layers MLP with numpy
class NN(object):
    def __init__(self, hidden_dims = (450, 400),  # (number of paras between 0.5M and 1M)
                 n_hidden = 2, mode = "train",
                 datapath = None, model_path = None):
        
        # Weights list
        self.weights = []
        # Bias list
        self.bias = []
        
        # Init weights and bias of first layer
        self.weights.append(np.empty((hidden_dims[0], N_INPUT)))
        self.bias.append(np.zeros((hidden_dims[0])))
        
        # Init weights and bias for hidden layers
        for i in range(n_hidden - 1):
            self.weights.append(np.empty((hidden_dims[i+1], hidden_dims[i])))
            self.bias.append(np.zeros((hidden_dims[i+1])))
        
        # Init weights and bias of output layer
        self.weights.append(np.empty((N_CLASSES, hidden_dims[-1])))
        self.bias.append(np.zeros((N_CLASSES)))

    def initialize_weights(self, method = "zero"):
        # Init weights in order of layers
        for idx, weight in enumerate(self.weights):
            if method == "zero":
                self.weights[idx] = np.zeros(shape = weight.shape)
            elif method == "normal":
                self.weights[idx] = np.random.normal(loc = 0, scale = 1, size = weight.shape)
            elif method == "glorot":
                d = np.sqrt(6 / (weight.shape[0] + weight.shape[1]))
                self.weights[idx] = np.random.uniform(low = -d, high = d, size = weight.shape)
    
    def forward(self, input):
        self.cache = [input]
        for weight, bias in zip(self.weights, self.bias):
            self.cache.append(self.activation(np.dot(weight, self.cache[-1]) + bias))
        last_cache = self.cache.pop()
        return self.softmax(last_cache)

    def activation(self, input):
        return np.maximum(0, input)

    def loss(self, prediction, label):
        return -(np.log(self.softmax(prediction)[label]))

    def softmax(self, input):
        max_input = np.max(input)
        return np.exp(input - max_input) / np.sum(np.exp(input - max_input))

    def backward(self, output, label):
        tmp_list = []
        for idx, o in enumerate(output):
            if idx == label:
                tmp_list.append(o - 1)
            else:
                tmp_list.append(o)
        pre_activation_grads = np.asarray(tmp_list).reshape(-1, 1)
        
        # Weight gradients 
        self.weight_grads = []
        # Bias gradients
        self.bias_grads = []
        
        for idx, (weight, _) in enumerate(zip(reversed(self.weights), reversed(self.bias))):
            layer_passed = np.asarray(list(reversed(self.cache))[idx])
            self.weight_grads.insert(0, np.dot(pre_activation_grads, layer_passed.reshape(-1, 1).T))
            self.bias_grads.insert(0, pre_activation_grads.reshape(-1))
            layer_passed_grads = np.dot(weight.T, pre_activation_grads)
            
            tmp_list = []
            for cached in list(reversed(self.cache))[idx]:
                if cached > 0:
                    tmp_list.append(1)
                else:
                    tmp_list.append(0)
            pre_activation_grads = layer_passed_grads * np.asarray(tmp_list).reshape(-1, 1)

    def update(self, learning_rate):
        for idx, (weight_grad, bias_grad) in enumerate(zip(self.weight_grads, self.bias_grads)):
            self.weights[idx] = self.weights[idx] - learning_rate * weight_grad
            self.bias[idx] = self.bias[idx] - learning_rate * bias_grad
    
    def test(self, inputs, labels):
        loss, acc = zip(*[(self.loss(self.forward(input), label), np.argmax(self.forward(input)) == label)
                          for input,label in zip(inputs, labels)])
        return np.mean(loss), np.mean(acc)
    
    def train(self, inputs, labels, N_epochs = 1, lr = 0.001, log = True):
        losses = []
        for epoch in range(N_epochs):
            loss = []
            for idx, (input, label) in enumerate(zip(inputs, labels), 1):
                result = self.forward(input)
                loss.append(self.loss(result, label))
                if idx % 500 == 0 and log:
                    print("Epoch{:2d} - {:.3f}".format(epoch + 1, np.mean(loss)))
                self.backward(result, label)
                self.update(lr)
            losses.append(np.mean(loss))
        return losses

### 2. Initialization

#### Zero

In [ ]:
nn = NN()
nn.initialize_weights("zero")
zero_loss = nn.train(X_train, Y_train, N_epochs = 10, lr = 1e-3)

#### Normal

In [ ]:
nn = NN()
nn.initialize_weights("normal")
normal_loss = nn.train(X_train, Y_train, N_epochs = 10, lr = 1e-3)

#### Glorot

In [ ]:
nn = NN()
nn.initialize_weights("glorot")
glorot_loss = nn.train(X_train, Y_train, N_epochs = 10, lr = 1e-3)

#### Comparison of loss curves

In [ ]:
from matplotlib import pyplot as plt

plt.plot(range(1, 11), zero_loss, label="zero", color='b')
plt.plot(range(1, 11), normal_loss, label="normal", color='c')
plt.plot(range(1, 11), glorot_loss, label="glorot", color='r')
plt.legend(title = "Initialization")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

### 3. Hyperparameter Search

In [ ]:

# Grid search for best hyper-parameters
def grid_search():
    best_nn = None
    best_acc = -1
    best_h1 = -1
    best_h2 = -1
    best_lr = -1
    
    for h1 in [450, 350, 150]:
        for h2 in [400, 200, 100]:
            for lr_item in [0.1, 0.01, 0.001]:
                nn = NN(hidden_dims = (h1, h2))
                nn.initialize_weights("glorot")
                nn.train(X_train, Y_train, N_epochs = 10, lr = lr_item, log = False)
                _, acc = nn.test(X_valid, Y_valid)
                print("h1:{0}, h2:{1}, lr:{2}, acc:{3}".format(h1, h2, lr_item, acc))
                if acc > best_acc:
                    best_acc = acc
                    best_nn = nn
                    best_h1 = h1
                    best_h2 = h2
                    best_lr = lr_item
                    
    return best_nn

_, acc = grid_search().test(X_test, Y_test)
print("Acc on the test set: {:.3%}".format(acc))